<a href="https://colab.research.google.com/github/Prothoma2001/Real-Time-Bangla-Sign-Language-Recognition/blob/main/Bangla_Sign_Language_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#dataset 2

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Define the path to your dataset directory
dataset_path = "/content/drive/MyDrive/BdSL_OPSA22_STATIC2"

# Define the dimensions of your images
img_width, img_height = 64,64

# Define the batch size and number of epochs for training
batch_size = 8
epochs = 10

# Initialize lists to hold the data and labels
data = []
labels = []

# Loop over the dataset directory
for i, class_folder in enumerate(os.listdir(dataset_path)):
    class_folder_path = os.path.join(dataset_path, class_folder)
    # Loop over the images in each class folder
    for image_name in os.listdir(class_folder_path):
        image_path = os.path.join(class_folder_path, image_name)
        # Load the image, resize to the desired dimensions, and convert to gray scale
        image = cv2.imread(image_path)
        image = cv2.resize(image, (img_width, img_height))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # fix
        # Add the pre-processed image and its label to the lists
        data.append(image)
        labels.append(i)

# Convert the data and labels lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data and labels into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.3, random_state=42)

# Normalize the pixel values to be between 0 and 1
train_data = train_data.astype("float32") / 255.0
test_data = test_data.astype("float32") / 255.0

# Convert the labels to one-hot encoded vectors
num_classes = len(np.unique(labels))
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

#creating model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=train_data.shape[1:]))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

model.save("OurMainModel_2.h5")

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/10
739/739 [==============================] - 21s 11ms/step - loss: 2.6580 - accuracy: 0.2889 - val_loss: 1.2477 - val_accuracy: 0.6212
Epoch 2/10
739/739 [==============================] - 8s 10ms/step - loss: 0.9949 - accuracy: 0.6991 - val_loss: 0.7823 - val_accuracy: 0.7559
Epoch 3/10
739/739 [==============================] - 7s 10ms/step - loss: 0.6139 - accuracy: 0.8014 - val_loss: 0.5820 - val_accuracy: 0.8191
Epoch 4/10
739/739 [==============================] - 8s 11ms/step - loss: 0.4317 - accuracy: 0.8633 - val_loss: 0.4142 - val_accuracy: 0.8665
Epoch 5/10
739/739 [==============================] - 8s 11ms/step - loss: 0.3129 - accuracy: 0.8989 - val_loss: 0.3823 - val_accuracy: 0.8898
Epoch 6/10
739/739 [==============================] - 8s 10ms/step - loss: 0.2414 - accuracy: 0.9258 - val_loss: 0.3448 - val_accuracy: 0.9103
Epoch 7/10
739/739 [==============================] - 7s 10ms/step - loss: 0.1838 - accuracy: 0.9409 - val_loss: 0.3617 - val_accuracy: 0.907

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Get the predicted class probabilities for the test set
y_pred = model.predict(test_data)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded test labels to class labels
y_true = np.argmax(test_labels, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Compute the classification report
class_report = classification_report(y_true, y_pred)

# Print the results
print("Confusion matrix:\n", conf_matrix)
print("Classification report:\n", class_report)

80/80 [==============================] - 1s 4ms/step
Confusion matrix:
 [[50  0  0 ...  0  0  0]
 [ 0 28  1 ...  0  0  0]
 [ 0  1 34 ...  0  0  0]
 ...
 [ 0  0  0 ... 46  0  1]
 [ 0  0  0 ...  0 29  0]
 [ 0  0  0 ...  1  0 69]]
Classification report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        54
           1       0.72      0.82      0.77        34
           2       0.97      0.87      0.92        39
           3       0.78      0.88      0.83        73
           4       0.95      0.95      0.95        59
           5       0.93      1.00      0.97        43
           6       0.84      0.97      0.90        37
           7       0.96      0.83      0.89        53
           8       0.78      0.90      0.84        20
           9       0.98      0.89      0.93        61
          10       0.96      0.96      0.96        55
          11       0.95      0.93      0.94        57
          12       0.87      0.90      0.88  

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(test_labels.argmax(axis=1), y_pred)

# Compute the sensitivity and specificity
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Sensitivity: 1.0
Specificity: 1.0


In [ ]:
predictions = model.predict(test_data)
test_labels = np.argmax(test_labels, axis=1)
predictions = np.argmax(predictions, axis=1)
from sklearn.metrics import classification_report
report = classification_report(test_labels, predictions)

from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(test_labels, predictions)

for i in range(num_classes):
    tp = confusion[i, i]
    fp = np.sum(confusion[:, i]) - tp
    fn = np.sum(confusion[i, :]) - tp
    tn = np.sum(confusion) - tp - fp - fn
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    print("Class {}: Sensitivity = {:.2f}, Specificity = {:.2f}, Accuracy = {:.2f}".format(i, sensitivity, specificity, accuracy)) 

80/80 [==============================] - 1s 6ms/step
Class 0: Sensitivity = 0.93, Specificity = 1.00, Accuracy = 1.00
Class 1: Sensitivity = 0.82, Specificity = 1.00, Accuracy = 0.99
Class 2: Sensitivity = 0.87, Specificity = 1.00, Accuracy = 1.00
Class 3: Sensitivity = 0.88, Specificity = 0.99, Accuracy = 0.99
Class 4: Sensitivity = 0.95, Specificity = 1.00, Accuracy = 1.00
Class 5: Sensitivity = 1.00, Specificity = 1.00, Accuracy = 1.00
Class 6: Sensitivity = 0.97, Specificity = 1.00, Accuracy = 1.00
Class 7: Sensitivity = 0.83, Specificity = 1.00, Accuracy = 1.00
Class 8: Sensitivity = 0.90, Specificity = 1.00, Accuracy = 1.00
Class 9: Sensitivity = 0.89, Specificity = 1.00, Accuracy = 1.00
Class 10: Sensitivity = 0.96, Specificity = 1.00, Accuracy = 1.00
Class 11: Sensitivity = 0.93, Specificity = 1.00, Accuracy = 1.00
Class 12: Sensitivity = 0.90, Specificity = 1.00, Accuracy = 1.00
Class 13: Sensitivity = 0.94, Specificity = 1.00, Accuracy = 1.00
Class 14: Sensitivity = 0.69, Spe

In [ ]:
#dataset 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Define the path to your dataset directory
dataset_path = "/content/drive/MyDrive/BdSL_OPSA22_STATIC1"

# Define the dimensions of your images
img_width, img_height = 64,64

# Define the batch size and number of epochs for training
batch_size = 8
epochs = 10

# Initialize lists to hold the data and labels
data = []
labels = []

# Loop over the dataset directory
for i, class_folder in enumerate(os.listdir(dataset_path)):
    class_folder_path = os.path.join(dataset_path, class_folder)
    # Loop over the images in each class folder
    for image_name in os.listdir(class_folder_path):
        image_path = os.path.join(class_folder_path, image_name)
        # Load the image, resize to the desired dimensions, and convert to gray scale
        image = cv2.imread(image_path)
        image = cv2.resize(image, (img_width, img_height))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # fix
        # Add the pre-processed image and its label to the lists
        data.append(image)
        labels.append(i)

# Convert the data and labels lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data and labels into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.3, random_state=42)

# Normalize the pixel values to be between 0 and 1
train_data = train_data.astype("float32") / 255.0
test_data = test_data.astype("float32") / 255.0

# Convert the labels to one-hot encoded vectors
num_classes = len(np.unique(labels))
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

#creating model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=train_data.shape[1:]))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))

model.save("OurMainModel_2.h5")

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/10
2135/2135 [==============================] - 35s 11ms/step - loss: 2.0438 - accuracy: 0.4456 - val_loss: 0.3764 - val_accuracy: 0.8841
Epoch 2/10
2135/2135 [==============================] - 21s 10ms/step - loss: 0.3924 - accuracy: 0.8769 - val_loss: 0.1965 - val_accuracy: 0.9351
Epoch 3/10
2135/2135 [==============================] - 24s 11ms/step - loss: 0.2427 - accuracy: 0.9211 - val_loss: 0.1578 - val_accuracy: 0.9481
Epoch 4/10
2135/2135 [==============================] - 24s 11ms/step - loss: 0.1949 - accuracy: 0.9355 - val_loss: 0.1358 - val_accuracy: 0.9568
Epoch 5/10
2135/2135 [==============================] - 21s 10ms/step - loss: 0.1558 - accuracy: 0.9472 - val_loss: 0.0926 - val_accuracy: 0.9749
Epoch 6/10
2135/2135 [==============================] - 21s 10ms/step - loss: 0.1386 - accuracy: 0.9530 - val_loss: 0.1085 - val_accuracy: 0.9675
Epoch 7/10
2135/2135 [==============================] - 24s 11ms/step - loss: 0.1210 - accuracy: 0.9606 - val_loss: 0.0912 -

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Get the predicted class probabilities for the test set
y_pred = model.predict(test_data)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded test labels to class labels
y_true = np.argmax(test_labels, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Compute the classification report
class_report = classification_report(y_true, y_pred)

# Print the results
print("Confusion matrix:\n", conf_matrix)
print("Classification report:\n", class_report)

229/229 [==============================] - 1s 4ms/step
Confusion matrix:
 [[140   0   0 ...   0   0   0]
 [  0  62   0 ...   0   0   0]
 [  0   0 101 ...   0   0   0]
 ...
 [  0   0   0 ... 129   0   0]
 [  0   0   0 ...   0 164   0]
 [  0   0   0 ...   0   0 126]]
Classification report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       142
           1       0.97      0.78      0.87        79
           2       1.00      0.99      1.00       102
           3       0.95      0.96      0.95        73
           4       0.90      0.99      0.94       109
           5       0.94      0.83      0.88        41
           6       0.98      0.97      0.98       156
           7       0.94      0.94      0.94        63
           8       1.00      1.00      1.00       121
           9       0.98      0.99      0.99       123
          10       0.81      0.85      0.83        87
          11       0.98      0.93      0.95       169
       

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(test_labels.argmax(axis=1), y_pred)

# Compute the sensitivity and specificity
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Sensitivity: 1.0
Specificity: 1.0


In [ ]:
predictions = model.predict(test_data)
test_labels = np.argmax(test_labels, axis=1)
predictions = np.argmax(predictions, axis=1)
from sklearn.metrics import classification_report
report = classification_report(test_labels, predictions)

from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(test_labels, predictions)

for i in range(num_classes):
    tp = confusion[i, i]
    fp = np.sum(confusion[:, i]) - tp
    fn = np.sum(confusion[i, :]) - tp
    tn = np.sum(confusion) - tp - fp - fn
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    print("Class {}: Sensitivity = {:.2f}, Specificity = {:.2f}, Accuracy = {:.2f}".format(i, sensitivity, specificity, accuracy)) 

229/229 [==============================] - 1s 4ms/step
Class 0: Sensitivity = 0.99, Specificity = 1.00, Accuracy = 1.00
Class 1: Sensitivity = 0.78, Specificity = 1.00, Accuracy = 1.00
Class 2: Sensitivity = 0.99, Specificity = 1.00, Accuracy = 1.00
Class 3: Sensitivity = 0.96, Specificity = 1.00, Accuracy = 1.00
Class 4: Sensitivity = 0.99, Specificity = 1.00, Accuracy = 1.00
Class 5: Sensitivity = 0.83, Specificity = 1.00, Accuracy = 1.00
Class 6: Sensitivity = 0.97, Specificity = 1.00, Accuracy = 1.00
Class 7: Sensitivity = 0.94, Specificity = 1.00, Accuracy = 1.00
Class 8: Sensitivity = 1.00, Specificity = 1.00, Accuracy = 1.00
Class 9: Sensitivity = 0.99, Specificity = 1.00, Accuracy = 1.00
Class 10: Sensitivity = 0.85, Specificity = 1.00, Accuracy = 1.00
Class 11: Sensitivity = 0.93, Specificity = 1.00, Accuracy = 1.00
Class 12: Sensitivity = 0.91, Specificity = 1.00, Accuracy = 1.00
Class 13: Sensitivity = 0.98, Specificity = 1.00, Accuracy = 1.00
Class 14: Sensitivity = 0.99, S

In [ ]:
#Check model summary

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


#creating model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64,64,3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(46, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 64)        36928     
                                                      